In [ ]:
root_dir = !git rev-parse --show-toplevel
import os; os.chdir(root_dir[0])

In [ ]:
BASE_URL = 'http://www.atlasbrasil.org.br'

In [ ]:
HEADERS = ''' 
    -H 'Connection: keep-alive'
    -H 'Cache-Control: max-age=0'
    -H 'Upgrade-Insecure-Requests: 1'
    -H 'User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'
    -H 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3'
    -H 'Referer: http://www.atlasbrasil.org.br/2013/pt/consulta/'
    -H 'Accept-Language: en-US,en;q=0.9,pt-BR;q=0.8,pt;q=0.7'
'''.replace('\n', ' ')

In [ ]:
!curl '{BASE_URL}/2013/pt/download/' {HEADERS} --insecure > /tmp/a.html

In [ ]:
import bs4

In [ ]:
soup = bs4.BeautifulSoup(open('/tmp/a.html').read())

In [ ]:
BASE_PATH = soup.find('head').find('base').attrs['href']
BASE_PATH

In [ ]:
downloads = soup.find_all(class_='blue_button')
links = [b.parent.attrs['href'] for b in downloads]
assert  links[0] == 'data/rawData/Indicadores Atlas - RADAR IDHM.xlsx'

In [ ]:
!mkdir -p data/raw

In [ ]:
import urllib
for l in links[:]:
    url = BASE_URL + BASE_PATH + urllib.parse.quote(l)
    print(url)
    file = l.split('/')[-1]
    !cd data/raw/; curl "{url}" {HEADERS} --insecure -o '{file}'

In [ ]:
# !head data/raw/Indicadores\ Atlas\ -\ RADAR\ IDHM.xlsx 

In [ ]:
lines = !ls -1 data/raw/*.zip | wc --lines
size = !du -s data/raw/

assert int(size[0].split('\t')[0]) > 10000, size
assert lines[0] == '24', lines


## Unzippin all this crap

In [ ]:
%%bash
rm -rf data/preprocessed
mkdir -p data/preprocessed
mkdir -p data/preprocessed/others
cp data/raw/* data/preprocessed/
shopt -s extglob
cd data/preprocessed/ && mv !(dados_*.zip|others) others 

In [ ]:
%%bash
set -e
cd data/preprocessed/
unp *.zip > /dev/null 2>&1
find . -mindepth 2 -type f -exec mv -t . '{}' +
yes | unp *.rar > /dev/null 2>&1
find . -mindepth 2 -type f -exec mv -t . '{}' +
rm *.zip *.rar
find . -mindepth 1 -type d -exec rmdir '{}' +
echo done

In [ ]:
%%bash
set -e
cd data/preprocessed/
mkdir udh; mkdir regional; mkdir regiao_metro;
mv *UDH* udh/
mv *Regional* regional/
mv *REGIO* regional/
mv *RM* regiao_metro/

In [ ]:
%%bash
exit 0
## Delete all non UDH files
set -e
cd data/preprocessed/udh/
mkdir die; mkdir save;
mv *UDH* save/
mv * die/ > /dev/null 2>&1 || true
mv die/save .
rm -r die
mv save/* .
rmdir save

## Check

In [ ]:
lines = !ls -1 data/preprocessed/udh/*.shp | wc --lines
size = !du -s data/raw/

assert int(size[0].split('\t')[0]) > 10000, size
assert lines[0] == '24', lines

# Convert Excels to parquets

In [ ]:
import glob
excels = glob.glob('data/preprocessed/udh/*.xlsx')
excels

In [ ]:
excels.remove('data/preprocessed/udh/agrupamento_UDHs_para_calculo_no_IBGE_2000_2010.xlsx')

In [ ]:
import multiprocessing, glob, pandas as pd
import pyarrow.lib

In [ ]:
pool = multiprocessing.Pool()
def convert_to_parquet(path):
    print(f'converting {path}')
    dfs = pd.read_excel(path, sheet_name=None)
    for sheet_name, df in dfs.items():
        try:
            if df.empty: continue
            df.to_parquet(path.replace('.xlsx', '__' + sheet_name + '.parquet'), index=False)
        except (Exception, pyarrow.lib.ArrowTypeError) as e:
            print('error in:')
            print((path, sheet_name, e))
res = list(pool.map(convert_to_parquet, excels))
del pool

In [ ]:
!du -sh data/preprocessed/udh/*parquet